NOTE: this notebook has been directly copied and pilfered from here: https://colab.research.google.com/drive/1Qg6E5kmf9n4G8rc5lXHIM_cQzMUFGH-g

# ML-Agents Open a UnityEnvironment
<img src="https://github.com/Unity-Technologies/ml-agents/blob/release_1/docs/images/image-banner.png?raw=true" align="middle" width="435"/>

## Setup

### Installing ml-agents

In [1]:
import mlagents

## Run the Environment

In [2]:
#@title Select Environment { display-mode: "form" }
env_id = "FoodCollector" #@param ['Basic', '3DBall', '3DBallHard', 'GridWorld', 'Hallway', 'VisualHallway', 'CrawlerDynamicTarget', 'CrawlerStaticTarget', 'Bouncer', 'SoccerTwos', 'PushBlock', 'VisualPushBlock', 'WallJump', 'Tennis', 'Reacher', 'Pyramids', 'VisualPyramids', 'Walker', 'FoodCollector', 'VisualFoodCollector', 'StrikersVsGoalie', 'WormStaticTarget', 'WormDynamicTarget']


### Start Environment from the registry

In [3]:
# -----------------
# This code is used to close an env that might not have been closed before
try:
  env.close()
except:
  pass
# -----------------

from mlagents_envs.registry import default_registry

env = default_registry[env_id].make()

  Extracting  FoodCollector : |████████████████████| 100% 
  Cleaning up FoodCollector : |████████████████████| 100% 


### Reset the environment
To reset the environment, simply call `env.reset()`. This method takes no argument and returns nothing but will send a signal to the simulation to reset.

In [4]:
env.reset()

### Behavior Specs


#### Get the Behavior Specs from the Environment

In [5]:
# We will only consider the first Behavior
behavior_name = list(env.behavior_specs)[0] 
print(f"Name of the behavior : {behavior_name}")
spec = env.behavior_specs[behavior_name]

Name of the behavior : FoodCollector?team=0


#### Get the Observation Space from the Behavior Specs

In [6]:
# Examine the number of observations per Agent
print("Number of observations : ", len(spec.observation_shapes))

# Is there a visual observation ?
# Visual observation have 3 dimensions: Height, Width and number of channels
vis_obs = any(len(shape) == 3 for shape in spec.observation_shapes)
print("Is there a visual observation ?", vis_obs)
spec.observation_shapes

Number of observations :  2
Is there a visual observation ? False


[(49,), (4,)]

#### Get the Action Space from the Behavior Specs

In [7]:
# Is the Action continuous or multi-discrete ?
if spec.is_action_continuous():
  print("The action is continuous")
if spec.is_action_discrete():
  print("The action is discrete")

# How many actions are possible ?
print(f"There are {spec.action_size} action(s)")

# For discrete actions only : How many different options does each action has ?
if spec.is_action_discrete():
  for action, branch_size in enumerate(spec.discrete_action_branches):
    print(f"Action number {action} has {branch_size} different options")
    


The action is discrete
There are 4 action(s)
Action number 0 has 3 different options
Action number 1 has 3 different options
Action number 2 has 3 different options
Action number 3 has 2 different options


### Steping the environment

#### Get the steps from the Environment
You can do this with the `env.get_steps(behavior_name)` method. If there are multiple behaviors in the Environment, you can call this method with each of the behavior's names.
_Note_ This will not move the simulation forward.

In [8]:
decision_steps, terminal_steps = env.get_steps(behavior_name)
print(decision_steps[0])
print(terminal_steps)

DecisionStep(obs=[array([0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.8472133 , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.93692917, 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.60939825, 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.9036788 , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.54550105,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.5921895 , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.5795279 ], dtype=float32), array([0., 0., 0., 0.], dtype=float32)], reward=0.0, agent_id=0, action_mask=[array([False, False, False]), array([False, False, False]), array([False, False, False]), array([False, False])])


#### Set actions for each behavior
You can set the actions for the Agents of a Behavior by calling `env.set_actions()` you will need to specify the behavior name and pass a tensor of dimension 2. The first dimension of the action must be equal to the number of Agents that requested a decision during the step.

In [9]:
empty_action = spec.create_empty_action(len(decision_steps))
env.set_actions(behavior_name, empty_action)
print(empty_action)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


#### Move the simulation forward
Call `env.step()` to move the simulation forward. The simulation will progress until an Agent requestes a decision or terminates.

In [10]:
env.step()

### Observations

#### Show the observations for one of the Agents
`DecisionSteps.obs` is a tuple containing all of the observations for all of the Agents with the provided Behavior name.
Each value in the tuple is an observation tensor containing the observation data for all of the agents.

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

for index, shape in enumerate(spec.observation_shapes):
  if len(shape) == 3:
    print("Here is the first visual observation")
    plt.imshow(decision_steps.obs[index][0,:,:,:])
    plt.show()

for index, shape in enumerate(spec.observation_shapes):
  if len(shape) == 1:
    print("First vector observations : ", decision_steps.obs[index][0,:])

First vector observations :  [0.         0.         1.         0.         0.         0.
 0.8472133  0.         0.         1.         0.         0.
 0.         0.93692917 0.         0.         1.         0.
 0.         0.         0.60939825 0.         0.         1.
 0.         0.         0.         0.9036788  0.         0.
 1.         0.         0.         0.         0.54550105 0.
 0.         0.         1.         0.         0.         0.5921895
 0.         0.         1.         0.         0.         0.
 0.5795279 ]
First vector observations :  [0. 0. 0. 0.]


### Run the Environment for a few episodes

In [ ]:
for episode in range(3):
  env.reset()
  decision_steps, terminal_steps = env.get_steps(behavior_name)
  tracked_agent = -1 # -1 indicates not yet tracking
  done = False # For the tracked_agent
  episode_rewards = 0 # For the tracked_agent
  while not done:
    # Track the first agent we see if not tracking 
    # Note : len(decision_steps) = [number of agents that requested a decision]
    if tracked_agent == -1 and len(decision_steps) >= 1:
      tracked_agent = decision_steps.agent_id[0] 

    # Generate an action for all agents
    action = spec.create_random_action(len(decision_steps))

    # Set the actions
    env.set_actions(behavior_name, action)

    # Move the simulation forward
    env.step()

    # Get the new simulation results
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    if tracked_agent in decision_steps: # The agent requested a decision
      episode_rewards += decision_steps[tracked_agent].reward
    if tracked_agent in terminal_steps: # The agent terminated its episode
      episode_rewards += terminal_steps[tracked_agent].reward
      done = True
  print(f"Total rewards for episode {episode} is {episode_rewards}")


Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data receiv

Total rewards for episode 0 is 0.0


Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data receiv

Total rewards for episode 1 is -2.0


Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.
Unknown side channel data received. Channel type: a1d8f7b7-cec8-50f9-b78b-d3e165a78520.


### Close the Environment to free the port it is using

In [ ]:
env.close()
print("Closed environment")